In [1]:
import pyvista as pv
import numpy as np
import traction_analysis as ta
import cProfile

AttributeError: module 'traction_analysis' has no attribute 'TractionAnalysis'

In [3]:
processor.process()

In [4]:
traction = np.array(processor.resampled_mesh.point_data["traction"])

In [5]:
traction

array([[-3.08170608e-06,  2.06801551e-05,  2.37370424e-05],
       [-4.13327021e-05,  4.31709515e-05, -3.09830814e-05],
       [-3.08170608e-06, -2.06801551e-05,  2.37370424e-05],
       ...,
       [ 1.07176180e-05,  3.10553290e-06,  2.48170759e-05],
       [ 1.20853295e-05,  1.62828601e-06,  2.45630620e-05],
       [ 1.08926617e-05,  1.55795513e-06,  2.55698581e-05]])

In [12]:
processor.calculate_traction_vec()

In [7]:
traction2 = np.array(processor.resampled_mesh.point_data["traction"])

In [8]:
traction2

array([[-3.08170608e-06,  2.06801551e-05,  2.37370424e-05],
       [-4.13327021e-05,  4.31709515e-05, -3.09830814e-05],
       [-3.08170608e-06, -2.06801551e-05,  2.37370424e-05],
       ...,
       [ 1.07176180e-05,  3.10553290e-06,  2.48170759e-05],
       [ 1.20853295e-05,  1.62828601e-06,  2.45630620e-05],
       [ 1.08926617e-05,  1.55795513e-06,  2.55698581e-05]])

In [9]:
traction2 - traction

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.69406589e-21, 0.00000000e+00, 0.00000000e+00]])

In [13]:
tract, dev, press = processor.integrate_forces()

In [14]:
tract2, dev2, press2 = processor.integrate_forces_vec()

In [15]:
tract2 - tract

array([0., 0., 0.])

In [16]:
dev2 - dev

array([-1.15948064e-10,  1.85860730e-11,  5.18701543e-11])

In [17]:
press2 - press

array([0., 0., 0.])